In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1，读入数据

In [ ]:
train = pd.read_csv('/kaggle/input/GiveMeSomeCredit/cs-training.csv',index_col=0)
test = pd.read_csv('/kaggle/input/GiveMeSomeCredit/cs-test.csv',index_col=0)
sample = pd.read_csv('/kaggle/input/GiveMeSomeCredit/sampleEntry.csv')

train.shape, test.shape, sample.shape

# 2， 填充缺失值

In [ ]:
train_default_dict = {} 
for attr in ['MonthlyIncome', 'NumberOfDependents']:
    print(attr, end='\t')
    median, mean, mode = train[attr].median(), train[attr].mean(), train[attr].mode()
    mode = mode.values
    if len(mode) == 1:
        mode = mode[0]
    train_default_dict[attr] = [median, mean, mode]
    print(median, mean, mode)
train_default_dict


In [ ]:
test_default_dict = {} 
for attr in ['MonthlyIncome', 'NumberOfDependents']:
    print(attr, end='\t')
    median, mean, mode = test[attr].median(), test[attr].mean(), test[attr].mode()
    mode = mode.values
    if len(mode) == 1:
        mode = mode[0]
    test_default_dict[attr] = [median, mean, mode]
    print(median, mean, mode)
test_default_dict


In [ ]:
train.MonthlyIncome.fillna(train_default_dict['MonthlyIncome'][0], inplace=True)
test.MonthlyIncome.fillna(train_default_dict['MonthlyIncome'][0], inplace=True)
train.NumberOfDependents.fillna(test_default_dict['NumberOfDependents'][1], inplace=True)
test.NumberOfDependents.fillna(test_default_dict['NumberOfDependents'][1], inplace=True)

train.isnull().sum(), '*'*50, test.isnull().sum()


# 3， 分离X_train, y_train, X_test

In [ ]:
X_train = train.iloc[:,1:].values
y_train = train.iloc[:,0].values
X_test = test.iloc[:,1:].values
X_train.shape, y_train.shape, X_test.shape

# 4, 分别归一化X_train, X_test 

In [ ]:
from sklearn import preprocessing

train_scaler = preprocessing.StandardScaler().fit(X_train)
print( train_scaler.mean_ , '\n'+'-'*50+'\n', train_scaler.scale_)
print('='*50)
test_scaler = preprocessing.StandardScaler().fit(X_test)
print( test_scaler.mean_ , '\n'+'-'*50+'\n', test_scaler.scale_)


In [ ]:
X_train_scaled = train_scaler.transform(X_train)
X_test_scaled = test_scaler.transform(X_test)

X_train_scaled.mean(axis=0), X_train_scaled.std(axis=0),   X_test_scaled.mean(axis=0), X_test_scaled.std(axis=0)


# 5，数据内容不再改动，模型堆叠

* 首先从X_train, y_train划分出训练集X_learn, y_learn; 验证集 X_valid, y_valid


In [ ]:
from sklearn.model_selection import train_test_split

X_learn, X_valid, y_learn, y_valid = train_test_split(X_train_scaled, y_train, random_state=0)
X_learn.shape, X_valid.shape, y_learn.shape, y_valid.shape

In [ ]:
#使用roc_auc 作为 metric
from sklearn.metrics import roc_auc_score

* 参考模型堆叠
* 参数来自于另一个notebook

## 5.1 试水

In [ ]:
from sklearn.ensemble import StackingClassifier
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier

estimators = [
    ('lgb', lgb.LGBMClassifier(n_estimators=54)),
    ('rfc', RandomForestClassifier(n_estimators=200)),
    ('mlp', MLPClassifier(hidden_layer_sizes=2)),
    ('knn', KNeighborsClassifier(n_neighbors=320, weights='distance', algorithm='auto'))
]

## 5.2 对stackClf调参

In [ ]:
from sklearn.model_selection import GridSearchCV

best_l, best_pt, maxauc = 0, 'none', 0
for hid_lay_siz in [1,2,3,4,5]:
    for pass_throu in [True, False]:
        reg = StackingClassifier(
            estimators=estimators,
            final_estimator=MLPClassifier(
                hidden_layer_sizes=hid_lay_siz,
                random_state=0
            ),
            passthrough=pass_throu,
            verbose=3
        )

        reg.fit(X_learn, y_learn)
        y_pred = reg.predict_proba(X_valid)[:,1]
        score = roc_auc_score(y_valid, y_pred)
        print(score)
        
        if score > maxauc:
            best_l, best_pt, maxauc = hid_lay_siz, pass_throu, score
print()
print(best_l, best_pt, maxauc)



In [ ]:
reg = StackingClassifier(
    estimators=estimators,
    final_estimator=MLPClassifier(
        hidden_layer_sizes=best_l,
        random_state=0
    ),
    passthrough=best_pt,
    verbose=3
)


reg.fit(X_train, y_train)
y_pred = reg.predict_proba(X_test)[:,1]


# 6 生成提交文件

In [ ]:
sample = pd.read_csv('../input/GiveMeSomeCredit/sampleEntry.csv')
sample

In [ ]:
sample['Probability'] = y_pred
sample.to_csv('./submit.csv',index=False)
reload = pd.read_csv('./submit.csv')
reload
